In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc =SparkContext(master="local", appName="New Spark Context")

In [3]:
data = sc.textFile("t8.shakespeare.txt").cache()

In [4]:
numAs = data.filter(lambda s: 'face' in s).count()
numBs = data.filter(lambda s: 'world' in s).count()
print("Line with 'face' : %i, lines with 'world': %i" %(numAs, numBs))


Line with 'face' : 489, lines with 'world': 688


In [5]:
sc

<SparkContext master=local appName=New Spark Context>

In [ ]:
pip install pyspark

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

In [8]:
import pandas as pd

In [9]:
df=spark.read.cv("creditcard.csv",header=True)

AttributeError: 'DataFrameReader' object has no attribute 'cv'

In [ ]:
df.printSchema()

In [ ]:
len(df.columns)

In [ ]:
df.select("Time").show(10)

In [ ]:
df.select(["V"+str(x) for x in range(1,5)]).show(10)

In [ ]:
df_pandas=df.toPandas()

In [ ]:
df_pandas.head(10)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(ncols=1,nrows=30,figsize=(5,45),sharey=False,sharex=True)
for i in range(30):
    ax[i].plot(df_pandas.iloc[:,i].value)
    #ax[i,0].plot(x=df.iloc[:,0].values-df.iloc[0,0].values,y=df_pandas.iloc[:,i].values)




In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassficationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler

In [ ]:
df.columns[1:-1] 

In [ ]:
from pyspark.sql.functions import col

for col name in df.columns[1:-1]+["class"]:
    df = df.withColumn(col_name, col(col_name).cast('float'))

In [ ]:
df=df.withColumnRenamed("Class","lable")

In [ ]:
df.columns

In [ ]:
vectorAssembler = VectorAssembler(inputCols = df.columns[1:-1],outputCol = 'feature')
df_tr = vectorAssembler.transform(df)
df_tr = df_tr.select(['feature','lable'])
df_tr.show(3)

In [10]:
df_tr.show(3,truncate=False)

NameError: name 'df_tr' is not defined

In [ ]:
df_tr.show(3)

In [ ]:
lr = LogisticRegression(maxIter=10,featuresCol="feature",lableCol="lable")

In [ ]:
#model=lr.fit(df_tr)

In [ ]:
#print(model.summary.areaUnderROC)

In [ ]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01])\
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

In [ ]:
crossval = CrossValidator(estimator=lr,
                         estimatorParamMaps=paramGrid,
                         evaluator=BinaryClassificationEvaluator(),
                         numFolds=2)

In [ ]:
cvModel = crossval.fit(df_tr)

In [ ]:
cvModel.avgMetrics

In [ ]:
df.printSchema()

In [ ]:
df.describe().show()

In [ ]:
df.columns

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols = df.columns[1:-1]